In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
import torchvision
import torchvision.transforms as transforms
import time
from PIL import Image

In [81]:
img = np.asarray(Image.open('Desktop/bottleFrames/frame_0002.png')).astype(np.uint8)
scale_w = img.shape[1] // 400
scale_h = img.shape[0] // 300
scaled = np.zeros((300, 400, img.shape[2]), dtype=img.dtype)
for i in range(300):
    for j in range(400):
        x = j * scale_w
        y = i * scale_h
        scaled[i, j] = img[y, x]
frame_N = scaled.copy()
frame_N = np.dot(frame_N[...,:3], [0.2126, 0.7152, 0.0722])

img = np.asarray(Image.open('Desktop/bottleFrames/frame_0001.png')).astype(np.uint8)
scale_w = img.shape[1] // 400
scale_h = img.shape[0] // 300
scaled = np.zeros((300, 400, img.shape[2]), dtype=img.dtype)
for i in range(300):
    for j in range(400):
        x = j * scale_w
        y = i * scale_h
        scaled[i, j] = img[y, x]
frame_N_1 = scaled.copy()
frame_N_1 = np.dot(frame_N_1[...,:3], [0.2126, 0.7152, 0.0722])

frame_N.shape

(300, 400)

In [82]:
def inter16x(sa):
    
    def ldps(sa_):
        safety = 1000
        vec = [0,0]
        while safety > 0:
            min_sad = [256**2, (1,1)]
            mid_x = sa_[0] + vec[0]
            mid_y = sa_[1] + vec[1]
            for mvx, mvy in ([0,1], [1,0], [0,-1], [-1,0], [0,0]):
                x_ = mid_x + mvx
                y_ = mid_y + mvy
                org_ = search_area[y_:y_+16, x_:x_+16]
                sad = int(sum(abs(org_.flatten() - curr_mb.flatten())))
                if sad <= min_sad[0]:
                    min_sad = [sad, (mvx, mvy)]
            if min_sad[1] == (0,0) or (x_ + min_sad[1][0] == 0 or y_ + min_sad[1][1] == 0) or (x_ + min_sad[1][0] > search_area.shape[0] - 17 or y_ + min_sad[1][1] > search_area.shape[1] - 17):
                return (min_sad[0], mid_x, mid_y)
            else:
                vec[0] += min_sad[1][0]
                vec[1] += min_sad[1][1]
            safety -= 1
        return (256**2, -1, -1)
    
    return ldps(sa)

In [83]:
motion_matrix = np.zeros((frame_N.shape[0] // 16 - 1, frame_N.shape[1] // 16 - 1), dtype=[('x', int), ('y', int)])
i_idx = -1
for i in range(16, frame_N.shape[0] - 16, 16):
    i_idx += 1
    j_idx = -1
    for j in range(16, frame_N.shape[1] - 16, 16):
        j_idx += 1
        curr_mb = frame_N[i:i+16, j:j+16]
        search_area = frame_N_1[i-12:i+28, j-12:j+28]
        m_x, m_y = inter16x((12,12))[1:3]
        motion_matrix[i_idx][j_idx] = (12 - m_x, 12 - m_y)
        
motion_matrix

array([[(  3,   6), (  9,   0), (  0,   4), (  0,   0), (  0,   4),
        (  6,   0), (  0,   5), (  1,  -5), (  0,   2), (  4,   2),
        (  4,   0), (  8,  -2), (  0,   0), ( -1,   0), ( -7,  -3),
        ( -7,   0), ( -6,   0), ( -3,  -7), ( -3,  -8), ( -8,   1),
        (  0,  -6), ( -7,  -3), (  0,  -5), (  0,   0)],
       [(  0,   3), (  0,   5), (  5,   1), (  0,   2), (  0,   0),
        (  0,   3), (  3,   4), (  5,  -6), (-10,   4), (  0,   4),
        (  0,   0), (  0,   0), (  0,   0), (  8,  -5), ( -2,  -3),
        ( -3,  -7), ( -6,  -6), ( -2,  -9), ( -3,  -8), ( -3,  -8),
        ( -2,  -8), ( -1,  -8), (-11,  -3), (  0,   0)],
       [(  0,   0), ( -2,   3), (  0,   3), (  0,   4), (  0,   2),
        (  0,   0), (  0,   3), (  6,   1), (  6,  -2), (  0,   1),
        (  3,   0), (  0,   0), (  0,   0), (  3,  -1), (  1,   3),
        ( -2,  -6), (-10,   0), ( -8,  -1), ( -3, -10), ( -3,  -9),
        ( -2,  -9), ( -2, -10), ( -3, -10), (  0,   0)],
       [(  0,